# OANDA REST-V20 API Data Fetcher

This notebook demonstrates how to:
1. List all available instruments from OANDA REST-V20 API
2. Download historical candle data for specified instruments from 2000 to now

## Setup

Before running this notebook:
1. Copy `.env.example` to `.env`
2. Fill in your OANDA API token and account ID in the `.env` file
3. Install requirements: `pip install -r requirements.txt`

In [33]:
from datetime import datetime
from datetime import timezone
import requests

%run ../scripts/fetch_data.py

# get_candles_all_prices('EUR_CHF', start_date='2009-01-01', end_date='2009-12-31', granularity='D') # Checked - daily works.
start_time = datetime(2019, 1, 1, 21, 0, tzinfo=timezone.utc)
end_time   = datetime(2019, 1, 2, 4, 0, tzinfo=timezone.utc)

# Fetch candles for the specified time range
# candles = _oanda_candles_request(requests.Session(), 'EUR_CHF', 'M1', start_time, end_time, price_type='M', count=None)
candles = get_candles_all_prices_during_interval('EUR_CHF', start_time, end_time, granularity='M1')
print(candles)

Fetching all prices of EUR_CHF [M1] from 2019-01-01 21:00:00+00:00 to 2019-01-02 03:00:00+00:00 ... got 246 rows.
Fetching all prices of EUR_CHF [M1] from 2019-01-02 03:00:00+00:00 to 2019-01-02 04:00:00+00:00 ... got 44 rows.
                               m_o      m_h      m_l      m_c  m_t      b_o  \
time                                                                          
2019-01-01 22:00:00+00:00  1.12566  1.12566  1.12566  1.12566    1  1.12511   
2019-01-01 22:01:00+00:00  1.12582  1.12582  1.12582  1.12582    1  1.12528   
2019-01-01 22:02:00+00:00  1.12586  1.12586  1.12586  1.12586    1  1.12532   
2019-01-01 22:03:00+00:00  1.12582  1.12582  1.12564  1.12564    4  1.12528   
2019-01-01 22:06:00+00:00  1.12558  1.12570  1.12558  1.12570    3  1.12504   
...                            ...      ...      ...      ...  ...      ...   
2019-01-02 03:54:00+00:00  1.12584  1.12586  1.12584  1.12584    3  1.12572   
2019-01-02 03:55:00+00:00  1.12586  1.12586  1.12584  1.12584 

In [1]:
import pandas as pd
big_file = '../data/processed/merged_data.parquet'
df = pd.read_parquet(big_file)
print("Big file loaded.")
y_col = 'm_realized_vol_5min_future'
# Drop rows with NaN in the target column - assume they are at the beginning of the dataset:
# find the first valid index
first_valid_index = df[y_col].first_valid_index()
df = df.loc[first_valid_index:]
print("NaNs dropped.")
df.to_parquet(big_file)
print("Processed data saved.")


Big file loaded.
NaNs dropped.
Processed data saved.


## Function: List All Available Instruments

In [18]:
def list_instruments():
    """
    List all instruments available in the OANDA account.
    
    Returns:
        pandas.DataFrame: DataFrame containing instrument details
    """
    url = f"{API_URL}/v3/accounts/{ACCOUNT_ID}/instruments"
    
    headers = {
        'Authorization': f'Bearer {API_TOKEN}',
        'Content-Type': 'application/json'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        instruments = data.get('instruments', [])
        
        # Create a DataFrame with relevant information
        instruments_data = []
        for inst in instruments:
            instruments_data.append({
                'name': inst.get('name'),
                'type': inst.get('type'),
                'displayName': inst.get('displayName'),
                'pipLocation': inst.get('pipLocation'),
                'displayPrecision': inst.get('displayPrecision'),
                'tradeUnitsPrecision': inst.get('tradeUnitsPrecision'),
                'minimumTradeSize': inst.get('minimumTradeSize'),
                'maximumTrailingStopDistance': inst.get('maximumTrailingStopDistance'),
                'minimumTrailingStopDistance': inst.get('minimumTrailingStopDistance')
            })
        
        df = pd.DataFrame(instruments_data)
        print(f"Found {len(df)} instruments")
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching instruments: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None

## Test: List All Instruments

In [19]:
# Get all available instruments
instruments_df = list_instruments()

if instruments_df is not None:
    # Display first few instruments
    print("\nFirst 10 instruments:")
    display(instruments_df.head(10))
    
    # Display instrument types
    print("\nInstrument types available:")
    print(instruments_df['type'].value_counts())
    
    # Display all instrument names
    print("\nAll instrument names:")
    print(instruments_df['name'].tolist())

Found 68 instruments

First 10 instruments:


,name,type,displayName,pipLocation,displayPrecision,tradeUnitsPrecision,minimumTradeSize,maximumTrailingStopDistance,minimumTrailingStopDistance
0,TRY_JPY,CURRENCY,TRY/JPY,-2,3,0,1,100.000,0.050
1,AUD_JPY,CURRENCY,AUD/JPY,-2,3,0,1,100.000,0.050
2,USD_CNH,CURRENCY,USD/CNH,-4,5,0,1,1.00000,0.00050
3,NZD_JPY,CURRENCY,NZD/JPY,-2,3,0,1,100.000,0.050
4,EUR_GBP,CURRENCY,EUR/GBP,-4,5,0,1,1.00000,0.00050
5,CHF_HKD,CURRENCY,CHF/HKD,-4,5,0,1,1.00000,0.00050
6,USD_CZK,CURRENCY,USD/CZK,-4,5,0,1,1.00000,0.00050
7,NZD_HKD,CURRENCY,NZD/HKD,-4,5,0,1,1.00000,0.00050
8,EUR_NOK,CURRENCY,EUR/NOK,-4,5,0,1,1.00000,0.00050
9,USD_CAD,CURRENCY,USD/CAD,-4,5,0,1,1.00000,0.00050



Instrument types available:
type
CURRENCY    68
Name: count, dtype: int64

All instrument names:
['TRY_JPY', 'AUD_JPY', 'USD_CNH', 'NZD_JPY', 'EUR_GBP', 'CHF_HKD', 'USD_CZK', 'NZD_HKD', 'EUR_NOK', 'USD_CAD', 'EUR_AUD', 'EUR_SGD', 'USD_HKD', 'CAD_HKD', 'USD_CHF', 'AUD_HKD', 'NZD_CHF', 'AUD_CHF', 'GBP_CHF', 'USD_THB', 'EUR_HKD', 'CHF_JPY', 'GBP_HKD', 'EUR_NZD', 'AUD_SGD', 'EUR_JPY', 'EUR_TRY', 'USD_JPY', 'SGD_JPY', 'GBP_ZAR', 'ZAR_JPY', 'USD_SEK', 'GBP_SGD', 'CAD_CHF', 'AUD_NZD', 'HKD_JPY', 'USD_NOK', 'GBP_AUD', 'USD_PLN', 'EUR_ZAR', 'NZD_USD', 'USD_ZAR', 'CAD_JPY', 'CAD_SGD', 'USD_HUF', 'EUR_CAD', 'CHF_ZAR', 'USD_DKK', 'EUR_HUF', 'EUR_CHF', 'EUR_DKK', 'EUR_USD', 'EUR_CZK', 'NZD_CAD', 'SGD_CHF', 'GBP_JPY', 'USD_TRY', 'GBP_PLN', 'AUD_USD', 'GBP_USD', 'USD_MXN', 'GBP_CAD', 'AUD_CAD', 'EUR_PLN', 'GBP_NZD', 'EUR_SEK', 'USD_SGD', 'NZD_SGD']


## Function: Download Candle Data

Downloads historical candle data for specified instruments from 2000 to now.

In [20]:
import cProfile
import pstats
from functools import wraps

def profile(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        profiler = cProfile.Profile()
        profiler.enable()
        result = func(*args, **kwargs)
        profiler.disable()
        stats = pstats.Stats(profiler).sort_stats('cumtime')
        print(f"\nProfiling results for {func.__name__}:")
        stats.print_stats(20)  # print top 20 lines
        return result
    return wrapper

# To profile a function, add @profile above its definition, e.g.:
# @profile
# def get_candles(...):
#     ...


In [21]:
OANDA_MAX_FREQ = 100 # Max 100 requests per second

def _ensure_utc(dt):
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def _parquet_filename(instrument, year, granularity, save_dir=None):
    fname = f"{instrument}_{year}{'_' + granularity if granularity != 'D' else ''}.parquet"
    if save_dir:
        fname = os.path.join(save_dir, fname)
    return fname

def _log_oanda_request(instrument, granularity, price_type, from_time, to_time, status_code, url, params):
    import datetime
    log_path = "oanda_requests.log"
    timestamp = datetime.datetime.now(datetime.timezone.utc).isoformat()
    log_line = f"[{timestamp}] [{instrument}] [{granularity}] [{price_type}] [{from_time}] [{to_time}] [{status_code}] [{url}] [{params}]\n"
    with open(log_path, "a") as f:
        f.write(log_line)

def _oanda_candles_request(session, instrument, granularity, current_start, end_dt, price_type='M', count=5000):
    """Low-level single request wrapper. Returns (candles, status_code, error_message)."""
    url = f"{API_URL}/v3/instruments/{instrument}/candles"
    headers = {
        'Authorization': f'Bearer {API_TOKEN}',
        'Content-Type': 'application/json'
    }
    params = {
        'from': current_start,
        'granularity': granularity,
        'price': price_type,
        'count': count
    }
    try:
        response = session.get(url, headers=headers, params=params)
        status_code = response.status_code
        _log_oanda_request(instrument, granularity, price_type, current_start, end_dt.isoformat(), status_code, url, params)
        response.raise_for_status()
        data = response.json()
        candles = data.get('candles', [])
        return candles, status_code, None
    except requests.exceptions.RequestException as e:
        status_code = getattr(e.response, 'status_code', 'ERR') if hasattr(e, 'response') and e.response is not None else 'ERR'
        _log_oanda_request(instrument, granularity, price_type, current_start, end_dt.isoformat(), status_code, url, params)
        error_message = str(e)
        if hasattr(e, 'response') and e.response is not None:
            error_message += f"\nResponse: {e.response.text}"
        return None, status_code, error_message

def _transform_candles(candles, price_type):
    rows = []
    for c in candles:
        base = {
            'time': c['time'], 
        }
        prefix, key = {'M': ('m', 'mid'), 'B': ('b', 'bid'), 'A': ('a', 'ask')}[price_type]
        base.update({ # 'm_o','m_h','m_l','m_c', etc.
            f'{prefix}_o': np.float32(c[key]['o']),
            f'{prefix}_h': np.float32(c[key]['h']),
            f'{prefix}_l': np.float32(c[key]['l']),
            f'{prefix}_c': np.float32(c[key]['c']),
            f'{prefix}_t': np.int32(c['volume'])
        })
        rows.append(base)
    if not rows:
        return None
    df = pd.DataFrame(rows)
    df['time'] = pd.to_datetime(df['time'], utc=True)
    df.set_index('time', inplace=True)
    df.sort_index(inplace=True)
    return df

def _fetch_price_type_paginated(instrument, start_dt, end_dt, granularity, price_type):
    """Generic pagination for a single price type (M,B,A)."""
    start_dt = _ensure_utc(start_dt); end_dt = _ensure_utc(end_dt)
    all_parts = []
    current_start = start_dt.isoformat()
    with requests.Session() as session:
        while True:
            candles, status_code, error_message = _oanda_candles_request(session, instrument, granularity, current_start, end_dt, price_type=price_type)
            if candles is None:
                if error_message:
                    print(f"Error fetching {price_type} candles for {instrument}: {error_message}")
                break
            if not candles:
                break
            part = _transform_candles(candles, price_type)
            if part is not None:
                all_parts.append(part)
            last_candle_time = max(c['time'] for c in candles)
            last_candle_dt = pd.to_datetime(last_candle_time, utc=True)
            if last_candle_dt > end_dt or len(candles) < 5000:
                break
            current_start = last_candle_time
            time.sleep(1.0 / OANDA_MAX_FREQ)
    if not all_parts:
        return None
    df = pd.concat(all_parts)
    df = df[(df.index >= start_dt) & (df.index <= end_dt)]
    return df

def fetch_candles(instrument, start_dt, end_dt, granularity, price_types=('M',)):
    """High-level fetch for one or multiple price types; returns merged DataFrame."""
    frames = []
    for pt in price_types:
        f = _fetch_price_type_paginated(instrument, start_dt, end_dt, granularity, pt)
        if f is not None:
            frames.append(f)
    if not frames:
        return None
    merged = frames[0]
    for add in frames[1:]:
        merged = merged.join(add, how='outer')
    merged.sort_index(inplace=True)
    return merged

def get_candles_all_prices(instrument, start_date='2005-01-01', end_date=None, granularity='S5', save_dir=None):
    start_dt = _ensure_utc(datetime.strptime(start_date, '%Y-%m-%d'))
    end_dt = _ensure_utc(datetime.strptime(end_date, '%Y-%m-%d')) if end_date else datetime.now(timezone.utc)
    if save_dir: os.makedirs(save_dir, exist_ok=True)
    for year in range(start_dt.year, end_dt.year + 1):
        year_start = datetime(year, 1, 1, tzinfo=timezone.utc)
        year_end = datetime(year, 12, 31, 23, 59, 59, tzinfo=timezone.utc)
        fetch_start = max(start_dt, year_start)
        fetch_end = min(end_dt, year_end)
        df = fetch_candles(instrument, fetch_start, fetch_end, granularity, price_types=('M','B','A'))
        if df is not None:
            fname = _parquet_filename(instrument, year, granularity + '_BA', save_dir)
            df.to_parquet(fname)
    return


## Test: Download Candle Data for Single Instrument

In [22]:
# Example: Download data for EUR_CHF
# get_candles_all_prices('EUR_CHF', start_date='2013-01-01', end_date='2013-12-31', granularity='S5')


## Test: Load data back from a Parquet file

In [ ]:
# Sanity load test after refactor
import pandas as pd
try:
    sample = pd.read_parquet('data/raw/EUR_CHF_2009_S5_BA.parquet')
    print('Loaded sample parquet rows:', len(sample))
    sample = sample[:5]
    all_columns = sample.columns.tolist()
    sample.drop(columns=[c for c in all_columns if c != 'm_t'], inplace=True)
    sample['since'] = sample.index.to_series().diff().dt.total_seconds().fillna(0) # Good, but would require completing with arithmetic sequence, once the index is completed.
    print(sample.head())
except Exception as e:
    print('Parquet load sanity check failed:', e)



Loaded sample parquet rows: 3020635
                           m_t  since
time                                 
2009-01-01 18:17:15+00:00    1    0.0
2009-01-01 18:27:35+00:00    1  620.0
2009-01-01 18:28:15+00:00    1   40.0
2009-01-01 18:28:25+00:00    1   10.0
2009-01-01 18:41:00+00:00    2  755.0


In [ ]:
#Question: are there missing data points in the downloaded data?
import pandas as pd
# Let's check all the data we have for NaNs
import os
data_dir = 'data/raw'
files = sorted([f for f in os.listdir(data_dir) if f.endswith('.parquet')])
print(f"Checking {len(files)} parquet files for missing data...")
for f in files:
    df = pd.read_parquet(os.path.join(data_dir, f))
    if df.isnull().values.any():
        missing_percentage = df.isnull().mean().mean() * 100
        print(f"Missing data found in file {f}: percentage of missing values: {missing_percentage:.2f}%")
    else:
        print(f"No missing data in file {f}.")

# Conclusion: Missing data found in 2005 (22%) and 2008 (0.03%); other years are clean.
# Decided: start analysis from 2009 onward (see features.py).

Checking 11 parquet files for missing data...
Missing data found in file EUR_CHF_2005_S5_BA.parquet: percentage of missing values: 22.47%
No missing data in file EUR_CHF_2006_S5_BA.parquet.
No missing data in file EUR_CHF_2007_S5_BA.parquet.
Missing data found in file EUR_CHF_2008_S5_BA.parquet: percentage of missing values: 0.58%
No missing data in file EUR_CHF_2009_S5_BA.parquet.
No missing data in file EUR_CHF_2010_S5_BA.parquet.
No missing data in file EUR_CHF_2011_S5_BA.parquet.
No missing data in file EUR_CHF_2012_S5_BA.parquet.
No missing data in file EUR_CHF_2013_S5_BA.parquet.
No missing data in file EUR_CHF_2014_S5_BA.parquet.
No missing data in file EUR_CHF_2015_S5_BA.parquet.


In [ ]:
# Question: do 'a_t, 'b_t', 'm_t' (number of quotes per candle) ever differ within the same candle?
import pandas as pd
# Let's check all the data we have for differences
import os
data_dir = 'data/raw'
files = sorted([f for f in os.listdir(data_dir) if f.endswith('.parquet')])
print(f"Checking {len(files)} parquet files for differences in 'a_t', 'b_t', 'm_t'...")
for f in files:
    df = pd.read_parquet(os.path.join(data_dir, f))
    diffs = df[(df['a_t'] != df['b_t']) | (df['a_t'] != df['m_t']) | (df['b_t'] != df['m_t'])]
    if not diffs.empty:
        print(f"Differences found in file {f} in {len(diffs)} candles:")
        print(diffs)   
    else:   
        print(f"No differences in file {f}.") 

# Conclusion: No differences found in any year. All 'a_t', 'b_t', 'm_t' are identical per candle.
# Decided: keep only 'm_t' and drop 'a_t', 'b_t' to save space (see features.py).

Checking 7 parquet files for differences in 'a_t', 'b_t', 'm_t'...
No differences in file EUR_CHF_2009_S5_BA.parquet.
No differences in file EUR_CHF_2010_S5_BA.parquet.
No differences in file EUR_CHF_2011_S5_BA.parquet.
No differences in file EUR_CHF_2012_S5_BA.parquet.
No differences in file EUR_CHF_2013_S5_BA.parquet.
No differences in file EUR_CHF_2014_S5_BA.parquet.
No differences in file EUR_CHF_2015_S5_BA.parquet.


Test the algorithm for time-since-last-true:

In [32]:
import pandas as pd
import numpy as np

# Create an artificial DataFrame for testing
df = pd.DataFrame({
    'a': [1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
    # 'b': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
})


def since_last_nonzero_nonshifted(series):
    mask = series != 0
    # cumsum increases at each nonzero, so group by this
    group = mask.cumsum()
    # Where group==0, it's before the first nonzero, so set to nan
    result = series.groupby(group).cumcount() + 1
    result[group == 0] = np.nan
    return result.values

def add_since_last_nonzero(df, col, new_col):
    df[new_col] = since_last_nonzero_nonshifted(df[col])
    df[new_col] = df[new_col].shift()
    return df

df = add_since_last_nonzero(df, 'a', 'since_a')
# df = add_since_last_nonzero(df, 'b', 'since_b')
print(df)

    a  since_a
0   1      NaN
1   0      1.0
2   0      2.0
3   0      3.0
4   1      4.0
5   0      1.0
6   1      2.0
7   0      1.0
8   1      2.0
9   1      1.0
10  1      1.0
11  0      1.0
12  0      2.0
13  1      3.0


## Summary

This notebook provides:
1. **`list_instruments()`** - Lists all instruments available in your OANDA account
2. **`get_candles_all_prices(instrument, start_date, end_date, granularity)`** - Downloads mid/bid/ask candle data for a single instrument

### Key Features:
- Handles OANDA's 5000 candle limit per request with automatic pagination
- Rate limiting protection with delays between requests
- Supports multiple granularities (daily, hourly, minute-level, etc.)
